## Load Library

In [1]:
import math
import time
import itertools

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

## Read File

In [2]:
inFile = open("tsp.txt", "r")

## Convert File to Pandas

In [3]:
arr_list = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    arr_list.append(line.split(" "))
    
df = pd.DataFrame(arr_list, columns = ["Name", "X", "Y"])

# Convert Value From String to int
df["X"] = df["X"].astype(int)
df["Y"] = df["Y"].astype(int)

df

,Name,X,Y
0,Depot,352,232
1,Point1,396,181
2,Parcel2,436,120
3,Parcel3,464,175
4,Point4,299,172
5,Parcel5,256,131
6,Parcel6,317,106
7,Point7,230,254
8,Parcel8,190,248
9,Parcel9,205,315


In [4]:
def distance(p1, p2):
    return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

In [5]:
class Point:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y
        
    def toString(self):
        return f"Name: {self.name}, X Pos: {self.x}, Y Pos: {self.y}"

In [6]:
def create_point(ind):
    row = df.iloc[ind, :]
    return Point(row.Name, row.X, row.Y)

In [7]:
def add_edge_to_graph(G, e1, e2, w):
    G.add_edge(e1, e2, weight=w)

## Create Point Array

In [8]:
pt_arr = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    tmp_arr = line.split(" ")
    
    pt_name = tmp_arr[0]
    x_pos = int(tmp_arr[1])
    y_pos = int(tmp_arr[2])
    
    pt = Point(pt_name, x_pos, y_pos)
    
    pt_arr.append(pt)
    
name_arr = list(map(lambda x : x.name, pt_arr))
name_arr

['Depot',
 'Point1',
 'Parcel2',
 'Parcel3',
 'Point4',
 'Parcel5',
 'Parcel6',
 'Point7',
 'Parcel8',
 'Parcel9',
 'Point10',
 'Parcel11',
 'Parcel12',
 'Point13',
 'Parcel14',
 'Parcel15',
 'Point16',
 'Parcel17',
 'Parcel18',
 'Point19',
 'Parcel20',
 'Parcel21',
 'Point22',
 'Parcel23',
 'Parcel24']

## Create Edge Array

In [9]:
adj_mat = []

name_arr = list(map(lambda x : x.name, pt_arr))

dist_val = 0

for pt in pt_arr:
    
    name = pt.name
    
    tmp_arr = []
    
    for pt2 in pt_arr:
        dist_val = round(distance(pt, pt2), 3)
        if pt.name == pt2.name or ("Depot" in pt.name and "Point" in pt2.name) or ("Point" in pt.name and "Point" in pt2.name):
            dist_val = float('inf')
        tmp_arr.append(dist_val)
    
    adj_mat.append(tmp_arr)
    
adj_mat

[[inf,
  inf,
  140.0,
  125.67,
  inf,
  139.345,
  130.771,
  inf,
  162.788,
  168.814,
  inf,
  207.509,
  205.419,
  inf,
  137.594,
  143.283,
  inf,
  221.278,
  261.297,
  inf,
  299.06,
  271.647,
  inf,
  179.471,
  213.242],
 [67.357,
  inf,
  72.945,
  68.264,
  inf,
  148.661,
  108.931,
  inf,
  216.622,
  233.317,
  inf,
  183.087,
  210.874,
  inf,
  204.482,
  201.109,
  inf,
  159.327,
  207.511,
  inf,
  258.94,
  248.244,
  inf,
  140.118,
  150.416],
 [140.0,
  72.945,
  inf,
  61.717,
  146.537,
  180.336,
  119.821,
  245.748,
  277.308,
  302.301,
  167.487,
  192.094,
  247.008,
  224.29,
  277.395,
  271.43,
  124.888,
  108.019,
  169.859,
  216.696,
  239.71,
  251.746,
  67.676,
  117.89,
  84.0],
 [125.67,
  68.264,
  61.717,
  inf,
  165.027,
  212.603,
  162.388,
  246.976,
  283.558,
  294.416,
  109.165,
  132.246,
  186.011,
  191.471,
  254.018,
  233.754,
  86.093,
  95.964,
  139.302,
  168.885,
  193.83,
  194.281,
  129.139,
  174.668,
  141.792]

In [10]:
def gen_adj_mat(adj_mat, column_arr):
    tmp_adj_mat = []
    for row_ind in range(len(adj_mat)):
        if row_ind in column_arr:
            tmp_list = []
            for col_ind in column_arr:
                tmp_list.append(adj_mat[row_ind][col_ind])
            tmp_adj_mat.append(tmp_list)
            
    return tmp_adj_mat

In [11]:
gen_adj_mat(adj_mat, [0, 2, 3, 4, 5])

[[inf, 140.0, 125.67, inf, 139.345],
 [140.0, inf, 61.717, 146.537, 180.336],
 [125.67, 61.717, inf, 165.027, 212.603],
 [80.056, 146.537, 165.027, inf, 59.414],
 [139.345, 180.336, 212.603, 59.414, inf]]

## Create Path

In [12]:
class Path:
    def __init__(self, path, cost):
        self.path = path
        self.cost = cost
        
    def __lt__(self, other):
        return self.cost <= other.cost
        
    def __str__(self):
        return f"Shortest Path:{self.path}\nMinimum Cost: {self.cost}"
    
    def __eq__(self, other):
        return self.path == other.path and self.cost == other.cost
    
    def getNodePath(self, name_arr):
        tmp_arr = list(map(lambda x : name_arr[x], self.path))
        return "->".join(tmp_arr)

## Create TSP Matrix

In [13]:
def held_karp(dists, column_arr):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            # Set bits for all nodes in this subset
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            
            # print(bits, subset)

            # Find the lowest cost to get to this subset
            for k in subset:
                prev = bits & ~(1 << k)

                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + dists[m][k], m))
                    
                # print((bits, k))
                C[(bits, k)] = min(res)

    # Calculate optimal cost
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res)

    # Backtrack to find full path
    path = ["Depot"] * (n + 1)
    for i in range(n - 1, 0, -1):
        tmp_val = parent
        path[i] = column_arr[tmp_val]
        _, parent = C[(bits, tmp_val)]
        bits = bits & ~(1 << tmp_val)
        
    return opt, path

In [14]:
# held_karp(adj_mat, name_arr)

In [15]:
# parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
# point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

# hk_path = Path([], float('inf'))

# final_start = time.process_time()

# combo_set = set(itertools.combinations(parcel_arr, len(point_arr)))
# total, ind = len(combo_set), 1

# print(f"Total Combinations: {total}")
# for parcel_set in combo_set:
    
#     tmp_list = [0] + [*parcel_set] + point_arr
#     tmp_list = sorted(tmp_list)
    
#     tmp_adj_mat = gen_adj_mat(adj_mat, tmp_list)
    
#     tmp_list = list(map(lambda x : name_arr[x], tmp_list))
    
#     start = time.process_time()
#     val, best_path = held_karp(tmp_adj_mat, tmp_list)
#     print(f"Time Taken (Held-Karp): {time.process_time() - start}")
    
#     hk_path = min(hk_path, Path(best_path, val))
    
#     print(f"Progress: {round(ind / total * 100.0, 2)}%\n")
    
#     ind += 1
    
# print(f"Final Time Taken (Held-Karp): {time.process_time() - final_start}")
# print(hk_path)

## Optimized Held Karp

In [16]:
# parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
# point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

# start = time.process_time()
# subset_arr, route_arr = [], []
# pN, n = len(point_arr), len(name_arr)
# for subset_size in range(2, 2 * pN):
#     for subset in itertools.combinations(range(1, n), subset_size):
#         subset_arr.append(subset)
        
# for subset in itertools.combinations(parcel_arr, len(point_arr)):
#     tmp_arr = sorted([*subset] + point_arr)
#     subset_arr.append(tuple(tmp_arr))
#     route_arr.append(sum([1 << elem for elem in tmp_arr]))
    
# print(subset_arr)

# print(time.process_time() - start)

In [17]:
# parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
# point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]

# start = time.process_time()

# subset_arr = []

# for k in range(1, len(point_arr) + 1):
#     for gSubset in itertools.combinations(parcel_arr, k):
#         for l in range(k - 1, k + 1):
#             if l == 0:
#                 continue
#             for rSubset in itertools.combinations(point_arr, l):
#                 subset_arr.append(([*gSubset], [*rSubset]))
            
                
# route_arr = []
# for subset in itertools.combinations(parcel_arr, len(point_arr)):
#     tmp_arr = sorted([*subset] + point_arr)
#     route_arr.append(sum([1 << elem for elem in tmp_arr]))
                
# subset_arr = sorted(subset_arr, key = lambda x : len(x[0]) + len(x[1]))
# print(subset_arr)

# print(route_arr)

In [18]:
def held_karp_2(dists, parcel_arr, point_arr, name_arr):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:                             
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}
    
    subset_arr, route_arr = [], []
    for k in range(1, len(point_arr) + 1):
        for gSubset in itertools.combinations(parcel_arr, k):
            for l in range(k - 1, k + 1):
                if l == 0:
                    continue
                for rSubset in itertools.combinations(point_arr, l):
                    subset_arr.append(([*gSubset], [*rSubset]))
                    
                    if len(gSubset) + len(rSubset) == 2 * len(point_arr):
                        bits = 0
                        for bit in gSubset:
                            bits |= 1 << bit
            
                        for bit in rSubset:
                            bits |= 1 << bit
                        
                        route_arr.append(bits)
                
    subset_arr = sorted(subset_arr, key = lambda x : len(x[0]) + len(x[1]))

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset in subset_arr:
        gSubset, rSubset = subset
        # Set bits for all nodes in this subset
        bits = 0
        
        for bit in gSubset:
            bits |= 1 << bit
            
        for bit in rSubset:
            bits |= 1 << bit
        
        subset2 = []
        
        if len(gSubset) == len(rSubset):
            # The last node FROM THE PREVIOUS SUBSET will be a red point
            subset2 = gSubset
        else:
            # The last node FROM THE PREVIOUS SUBSET will be a green point
            subset2 = rSubset
            
        subset1 = [0] * (len(gSubset) + len(rSubset))
        
        ind = 0
        for elem in gSubset:
            subset1[ind] = elem
            ind += 1
            
        for elem in rSubset:
            subset1[ind] = elem
            ind += 1

        # Find the lowest cost to get to this subset
        for k in subset1:
            prev = bits & ~(1 << k)

            opt, parent = float('inf'), 0
            for m in subset2:     
                # Check if Key Exist
                key = (prev, m)
                if key in C:
                    tmp_opt = C[key][0] + dists[m][k]
                    if tmp_opt <= opt:
                        opt = tmp_opt
                        parent = m
                
            C[(bits, k)] = (opt, parent)
                
    best_bits, best_opt, best_parent = 0, float('inf'), 0

    # Calculate optimal cost
    for route_bits in route_arr:
        for k in point_arr:
            tmp_opt = C[(route_bits,k)][0] + dists[k][0]
            
            if tmp_opt <= best_opt:
                best_opt = tmp_opt
                best_parent = k
                best_bits = route_bits

    # Backtrack to find full path
    path = ["Depot"] * (2 * len(point_arr) + 2)
    for i in range(2 * len(point_arr), 0, -1):
        tmp_val = best_parent
        path[i] = name_arr[tmp_val]
        _, best_parent = C[(best_bits, tmp_val)]
        best_bits = best_bits & ~(1 << tmp_val)
        
    return best_opt, path

In [19]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]
print(parcel_arr, point_arr)

[2, 3, 5, 6, 8, 9, 11, 12, 14, 15, 17, 18, 20, 21, 23, 24] [1, 4, 7, 10, 13, 16, 19, 22]


In [20]:
parcel_arr = [ind for (ind, val) in enumerate(name_arr) if "Parcel" in val]
point_arr = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]
start = time.process_time()
val, best_path = held_karp_2(adj_mat, parcel_arr, point_arr, name_arr)
hk2_path = Path(best_path, val)
print(hk2_path)
print(time.process_time() - start)

Shortest Path:['Depot', 'Parcel8', 'Point7', 'Parcel14', 'Point13', 'Parcel15', 'Point10', 'Parcel11', 'Point19', 'Parcel18', 'Point16', 'Parcel3', 'Point1', 'Parcel2', 'Point22', 'Parcel6', 'Point4', 'Depot']
Minimum Cost: 1417.3509999999999
66.8125


In [21]:
# print(hk_path == hk2_path)